In [2]:
%run helpers.ipynb

/home/moorad/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
(ratings,_,_) = load_ML_100k()

ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [117]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from multiprocessing import Pool, cpu_count
class EASE:
	def __init__(self, top_k = 10):
		self.top_k = top_k
		self.user_encoder = LabelEncoder()
		self.item_encoder = LabelEncoder()

	def fit(self, ratings, lambda_ = 0.5):
		users = self.user_encoder.fit_transform(ratings.loc[:,'user_id'])
		items = self.item_encoder.fit_transform(ratings.loc[:,'item_id'])
  
		values = np.ones(ratings.shape[0])
  
		X = csr_matrix((values, (users, items)))
  
		G = X.T.dot(X).toarray()
		diagIndices = np.diag_indices(G.shape[0])
		G[diagIndices] += lambda_
		P = np.linalg.inv(G)
		B = P / (-np.diag(P))
		B[diagIndices] = 0

		self.pred = X.dot(B)
  
	def predict(self, train, users, items):
		items = self.item_encoder.transform(items)
		dd = train.loc[train.user_id.isin(users)]
		dd['ci'] = self.item_encoder.transform(dd.item_id)
		dd['cu'] = self.user_encoder.transform(dd.user_id)
		display(dd)
		g = dd.groupby('cu')
		with Pool(cpu_count()) as p:
			user_preds = p.starmap(
				self.predict_for_user,
				[(user, group, self.pred[user, :], items, self.top_k) for user, group in g],
			)
		df = pd.concat(user_preds)
		df['item_id'] = self.item_encoder.inverse_transform(df['item_id'])
		df['user_id'] = self.user_encoder.inverse_transform(df['user_id'])
		return df

	@staticmethod
	def predict_for_user(user, group, pred, items, k):
		watched = set(group['ci'])
		candidates = [item for item in items if item not in watched]
		pred = np.take(pred, candidates)
		res = np.argpartition(pred, -k)[-k:]
		r = pd.DataFrame(
			{
				"user_id": [user] * len(res),
				"item_id": np.take(candidates, res),
				"score": np.take(pred, res),
			}
		).sort_values('score', ascending=False)
		return r

In [171]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, identity

from multiprocessing import Pool, cpu_count
class EASE2:
	def __init__(self, top_k = 10, lambda_ = 0.5):
		self.top_k = top_k
		self.lambda_ = lambda_
		# self.user_encoder = LabelEncoder()
		# self.item_encoder = LabelEncoder()

	def fit(self, training):
		users = training.user_id.values
		items = training.item_id.values
		ratings = training.rating.values
  
		values = np.ones(training.shape[0])
  
		X = csr_matrix((ratings, (users, items)))
  
		G = X.T.dot(X).toarray()
		diagIndices = np.diag_indices(G.shape[0])
		G[diagIndices] += lambda_
		P = np.linalg.inv(G)
		B = P / (-np.diag(P))
		B[diagIndices] = 0

		self.item_similarity = B # item_num * item_num
		self.item_similarity = np.array(self.item_similarity)
		self.interaction_matrix = X # user_num * item_num
  
	def predict(self, u, i):
		return self.interaction_matrix[u, :].multiply(self.item_similarity[:, i].T).sum(axis=1).getA1()[0]

	# def rank(self, test_loader):
	# 	rec_ids = None

	# 	for us, cands_ids in test_loader:
	# 		us = us.numpy()
	# 		cands_ids = cands_ids.numpy()

	# 		slims = np.expand_dims(self.interaction_matrix[us, :].todense(), axis=1) # batch * item_num -> batch * 1* item_num
	# 		sims = self.item_similarity[cands_ids, :].transpose(0, 2, 1) # batch * cand_num * item_num -> batch * item_num * cand_num
	# 		scores = np.einsum('BNi,BiM -> BNM', slims, sims).squeeze() # batch * 1 * cand_num -> batch * cand_num
	# 		rank_ids = np.argsort(-scores)[:, :self.topk]
	# 		rank_list = cands_ids[np.repeat(np.arange(len(rank_ids)).reshape(-1, 1), rank_ids.shape[1], axis=1), rank_ids]

	# 		rec_ids = rank_list if rec_ids is None else np.vstack([rec_ids, rank_list])

	# 	return rec_ids

	def full_rank(self, u):
		r = self.interaction_matrix[u, :] @ self.item_similarity
		scores = np.array(r).flatten()

		return np.argsort(-scores)[:self.topk]


In [172]:
from sklearn.model_selection import train_test_split
(ratings, movies, _) = load_ML_100k()

ratings = ratings.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
movies = movies.rename(columns={'movieId': 'item_id'})

(training, testing) = train_test_split(ratings, train_size=0.005)

training
training.shape[0]

504

In [186]:
# from logging import getLogger
from daisy.model.EASERecommender import EASE

config = {
    "gpu": "0",
	"UID_NAME": 'user_id',
	"IID_NAME": 'item_id',
	"INTER_NAME": 'rating',
	"user_num": 10,
	"item_num": 10,	
	"reg": 0.5,
	"topk": 10,
}

EASE(config)

# init_seed(config['seed'], config['reproducibility'])

# init_logger(config)
# logger = getLogger()
# logger.info(config)
# config['logger'] = logger


KeyError: 'logger'

In [173]:
model = EASE2()

model.fit(training)

MemoryError: Unable to allocate 242. GiB for an array with shape (180264, 180264) and data type float64

In [129]:
# user = training.iloc[0]

model.predict(training, testing.user_id.tolist(), testing.item_id.tolist())

ValueError: y contains previously unseen labels: [108, 137, 179, 183, 201, 220, 285, 295, 298, 310, 336, 386, 495, 547, 563, 573, 600, 632, 638, 698, 803, 876, 1040, 1054, 1116, 1137, 1144, 1156, 1163, 1180, 1337, 1423, 1427, 1440, 1458, 1473, 1496, 1514, 1565, 1574, 1661, 1675, 1679, 1752, 1771, 1807, 1824, 1825, 1836, 1870, 1926, 1932, 1980, 1985, 2037, 2055, 2056, 2098, 2175, 2180, 2304, 2362, 2368, 2400, 2415, 2451, 2462, 2503, 2538, 2544, 2559, 2563, 2573, 2586, 2597, 2611, 2652, 2674, 2698, 2743, 2744, 2754, 2757, 2765, 2789, 2800, 2824, 2837, 2848, 2862, 2865, 2886, 2887, 2969, 2979, 3010, 3014, 3043, 3056, 3073, 3094, 3120, 3280, 3284, 3315, 3344, 3345, 3379, 3389, 3459, 3514, 3531, 3586, 3619, 3622, 3640, 3652, 3658, 3716, 3720, 3739, 3789, 3806, 3851, 3870, 3888, 3895, 3912, 3914, 3919, 3942, 4064, 4123, 4143, 4166, 4174, 4189, 4236, 4284, 4338, 4355, 4374, 4410, 4454, 4471, 4484, 4495, 4497, 4522, 4615, 4635, 4645, 4658, 4671, 4675, 4749, 4783, 4804, 4813, 4852, 4877, 4926, 